In [13]:
import numpy as np
import sympy as sp

In [14]:
#Properties (N and mm)
E1 = 60800
E2 = 58250
v12 = 0.07
G12 = 4550
t = 0.21
v21 = v12 * E2 / E1

# Plate Parameters
a = 360
b = 360
m = 3
n = 3


**[Q] in the principal direction**  pg. 52 chap 2 of the three regions

In [7]:
Q_0 = np.array([[E1/(1-v12*v21), v12*E2/(1-v12*v21), 0],
                 [v12*E2/(1-v12*v21), E2/(1-v12*v21),0],
                 [0, 0, G12]])

**[Q] rotated in theta direction** pg. 53 and 54 of chap 2

In [9]:
layup = [(45,'f'),(0,'f'),(0,'f'),(45,'f'),(0,'f'),(0,'f'),(45,'f')]
Q_layup = []

#Layup assembly
for theta in layup:
    c = np.cos(theta[0]*np.pi/180)
    s = np.sin(theta[0]*np.pi/180)

    T_real = np.array([[c**2, s**2, 2*c*s],
                       [s**2, c**2, -2*c*s],
                       [-c*s, c*s, c**2-s**2]])

    T_engineering =  np.array([[c**2, s**2, c*s],
                               [s**2, c**2, -c*s],
                               [-2*c*s, 2*c*s, c**2-s**2]])


    if theta[1] == 'f':
        Q_layup.append((np.linalg.inv(T_real))@Q_0@T_engineering)


**[A], [B], [D]** pg. 24 fo chap 4 of the three regions

In [12]:
t_position = [-3.5*t, -2.5*t, -1.5*t, -0.5*t, 0.5*t, 1.5*t, 2.5*t, 3.5*t]

A = np.zeros(9).reshape(3,3)
B = np.zeros(9).reshape(3,3)
D = np.zeros(9).reshape(3,3)

for i in enumerate(Q_layup):
    zk1 = t_position[i[0]+1]
    zk0 = t_position[i[0]]
    
    A += (zk1 - zk0)*i[1]
    B += (1/2)*(zk1**2 - zk0**2)*i[1]
    D += (1/3)*(zk1**3 - zk0**3)*i[1]

### Rayleigh-Ritz formulation

In [24]:
x,y = sp.symbols(['x','y'])
Ny = sp.symbols('Ny')
xi, eta, r1, r2, t1, t2  = sp.symbols(['xi', 'eta', 'r1', 'r2', 't1', 't2'])

In [25]:
# Bardell terms

f_xi = {0: t1*(0.25*xi**3 - 0.75*xi + 0.5),
        1: r1*(0.125*xi**3 - 0.125*xi**2 - 0.125*xi + 0.125),
        2: t2*(-0.25*xi**3 + 0.75*xi + 0.5),
        3: r2*(-0.25*xi**3 + 0.75*xi + 0.5),
        4: xi**4/8 - xi**2/4 + 0.125,
        5: xi**5/8 - xi**3/4 + xi/8,
        6: 7*xi**6/48 - 5*xi**4/16 + 3*xi**2/16 - 0.0208333333333333,
        7: 3*xi**7/16 - 7*xi**5/16 + 5*xi**3/16 - xi/16,
        8: 33*xi**8/128 - 21*xi**6/32 + 35*xi**4/64 - 5*xi**2/32 + 0.0078125,
        9: 143*xi**9/384 - 33*xi**7/32 + 63*xi**5/64 - 35*xi**3/96 + 5*xi/128}

f_eta = {0: t1*(0.25*eta**3 - 0.75*eta + 0.5),
        1: r1*(0.125*eta**3 - 0.125*eta**2 - 0.125*eta + 0.125),
        2: t2*(-0.25*eta**3 + 0.75*eta + 0.5),
        3: r2*(-0.25*eta**3 + 0.75*eta + 0.5),
        4: eta**4/8 - eta**2/4 + 0.125,
        5: eta**5/8 - eta**3/4 + eta/8,
        6: 7*eta**6/48 - 5*eta**4/16 + 3*eta**2/16 - 0.0208333333333333,
        7: 3*eta**7/16 - 7*eta**5/16 + 5*eta**3/16 - eta/16,
        8: 33*eta**8/128 - 21*eta**6/32 + 35*eta**4/64 - 5*eta**2/32 + 0.0078125,
        9: 143*eta**9/384 - 33*eta**7/32 + 63*eta**5/64 - 35*eta**3/96 + 5*eta/128}

In [21]:
Nw = [[]]

for i in range(m):
    for j in range(n):
        Nw[0].append(sp.sin((i+1)*sp.pi*x/a)*sp.sin((j+1)*sp.pi*y/b))

Nw_values = sp.Matrix(Nw)

